In [ ]:
import glob
import math
import uproot
import numpy as np
from numpy.lib.function_base import meshgrid
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["svg"]

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

plt.style.use(hep.style.ROOT)  # For now ROOT defaults to CMS
# hep.set_style({"font.size":8})
# mpl.rc('font', size=8)

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

path1 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=3.0eV'
path2 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=4.0eV'
path3 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=6.1eV'
path4 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=8.2eV'
path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=11.5eV'

# glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(path + "/**.dat"))

li = []
cosphi_th = []
test_opening = []
allnames=[]
colnames = ["phi", "cos(theta)", "value"]

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  # r"\s+" is a regex (regular expression)
    # or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] # adding a column to df with the file name with no spaces and extension
    cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
    allnames.append(temp_name)
    df["filename"]=temp_name
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th = np.array(cosphi_th)
frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10: semantic order
frame_srt = frame.groupby("filename",sort=False)
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
#test initial plotting
counts_1_1 = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # values
counts_1_1 = counts_1_1.reshape(200,100) # reshape(rows=x,columns=y)

print("number of ROWS (x) = ", len(counts_1_1[:,1]), "number of COLUMNS (y for reshaping) = ", len(counts_1_1[1,:]), "type ", type(counts_1_1))

In [ ]:
# Single 2D maps from pandas without tranforming into numpy
frame.loc[frame["filename"] == "1_1"].plot.hexbin(x="phi", y="cos(theta)", C="value", sharex=False, gridsize=50)

In [ ]:
"""
##############################################NOTE####################################################
in elements tag:
cosphi_th=([[ 1,  1],   cosphi_th_cos=([[ 6,  1],       cosphi_phi=([[ 6,  1],      
            [ 1,  2],                   [ 6,  2],                    [ 5,  1],                   
            [ 1,  3],                   [ 6,  3],                    [ 4,  1],
            [ 1,  4],                   [ 6,  4],                    [ 3,  1],
            [ 1,  5],                   [ 6,  5],                    [ 2,  1],                   
            [ 1,  6],                   [ 6,  6],                    [ 1,  1],

in angles tag:
cosphi_adj_th=([[ 1,  -180],   cosphi_adj_cos=([[ -1,  -180],       cosphi_adj_phi=([[ -1,   -180],      
                [ 1,  -147],                    [ -1,  -147],                        [ -0.8, -180],                   
                [ 1,  -114],                    [ -1,  -114],                        [ -0.3, -180],
                [ 1,  -81],                     [ -1,  -81],                         [ 0.3,  -180],
                [ 1,  -49],                     [ -1,  -49],                         [ 0.8,  -180],                   
                [ 1,  -16],                     [ -1,  -16],                         [ 1,    -180],

cosphi_th has to be used JUST with counts_stemp!!

b1 maps cordinates. YOU SHOULD USE THIS AS ROTATION INPUT

list(zip(xgo_phi,ygo_phi))             list(zip(xgo_cos,ygo_cos))
[(-180.0, 1.0),                        [(-180.0, 1.0),
 (-180.0, 0.809),                       (-147.273, 1.0),
 (-180.0, 0.309),                       (-114.545, 1.0),
 (-180.0, -0.309),                      (-81.818, 1.0),
 (-180.0, -0.809),                      (-49.091, 1.0),
 (-180.0, -1.0),                        (-16.364, 1.0),
##############################################NOTE####################################################
######################################################################################################
                                    PHI - CTHETEA ELECTRON 
######################################################################################################
(alpha and beta in Philipp convention)
REMEBER: in Philipp simulations cos(theta)=[1,-1] becase theta=[0,np.pi]
SORTED = cos(theta)=[-1,1] IS NEVER USED for electrons coordinates
#NOTE: the following phi and ctheta are not unique: len=20000

"""
#SORTED FULL **cos(theta) = [-1,1]**, phi = [-179.1, 179.1]
# len = 200000
phi_full_sorted=frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["phi"] 
ctheta_full_sorted=frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["cos(theta)"]

#SORTED unique **cos(theta) = [-1,1]**, phi = [-179.1, 179.1]  
# len(phiMs) = 200, len(cosMs) = 100
phiMs=frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["phi"].unique()
cosMs=frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["cos(theta)"].unique()

#UNSORTED FULL phi = [-179.1, 179.1]
# len = 200000
phi_full = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
phi_rad = phi_full * np.pi/180.

#UNSORTED FULL cos(theta) = [1,-1]
# len = 200000
ctheta_full = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
theta_rad = np.arccos(ctheta_full)
theta_full = theta_rad*180./np.pi

#UNSORTED unique cos(theta) = [1,-1], phi = [-179.1, 179.1]
# len(phiMs) = 200, len(cosMs) = 100
#NOTE      np.any(phiM == phiMs) -> TRUE
#          np.any(cosM == np.flip(cosMs)) -> TRUE
phiM = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi = [-179.1, 179.1]
cosM = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta) = [1,-1]


#NOTE: for np.MESHGRID:
#for the 2D case with inputs of length M and N, the outputs are of shape (N, M) for ‘xy’ indexing and (M, N) for ‘ij’ indexing. Cartesia "xy" by default
#REMEBER: np.any(phiM == phiMs) -> TRUE

#This SHOULDN´T be use to inrepolate
#because **cos(theta) = [-1,1]**
phiMMs, cosMMs = np.meshgrid(phiMs, cosMs)
phiMMs_rad, thetaMMs_rad = np.meshgrid(phiMs*np.pi/180., np.arccos(cosMs))

#This is the right one to use to interpolate
# because cos(theta)=[1,-1]
phiMM, cosMM = np.meshgrid(phiM, cosM)
phiMM_rad, thetaMM_rad = np.meshgrid(phiM*np.pi/180., np.arccos(cosM))

print("pihM ", phiM.shape, "cosM ", cosM.shape)
print("pihMs ", phiMM.shape, "cosMs ", cosMM.shape)
print(type(phiM), type(phiMM))

In [ ]:
phiMM.T

In [ ]:
"""
##########################b1 map x,y coordiantes###############################
PHI - CTHETA photon with respect to the LF (in sperical coordinates)
NOTE the go coordiantes are organised accordinf to the SORTED MFPAD, therefore cos(theta)=[-1,1]
REDUCED=TRUE => ADJUSTED TO EXPERIMENT: -0.835,0.835,-165,165
REDUCED=FALSE => ADJUSTED TO THEORY: -1,1,-180,180
"""
# fugi = reload(fugi)

xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, reduced=False, source=True)
xgo_cos,ygo_cos,cosphi_adj_cos = fugi.create_gocoords(a=1, reduced=False, source=True)

# list(zip(xgo_phi,ygo_phi))
print("(xgo_cos (phi), ygo_cos (ctheta)) \n", list(zip(xgo_cos,ygo_cos))[:6])
print("\n cosphi_adj_cos  \n", cosphi_adj_cos[:6])
print("\n cosphi_adj_phi \n", cosphi_adj_phi[:6])

In [ ]:
"""
Creation of DataFrame to find the right element according to rotation
isuefull fot the 3d representation
NOTE allnames are UNSORTED: provide cosphi_df with cos(thea) = [1,-1] sorted by cos(theta)
NOTE all the coordinates are FULL!! this is iportant for rotation

according to automatic_72_CPR.f90 from philipp [1,6]: is from [0,P1] -> [cos(0), cos(PHI)] = [1,-1]
according to automatic_72_CPR.f90 from philipp [1,12]: is from [-PI,PI]

allnames  cos(theta)   phi
1_1       1            -180 
1_2       1            -147 
"""
cosphi_df_cos = np.around(np.array(list(itertools.product(np.cos(np.linspace(0,np.pi,6).tolist()),np.linspace(-180,180,12).tolist()))),3)
dfind=pd.DataFrame(cosphi_df_cos, columns=["ctheta","phi"])
dfind["item"]=allnames

#Sorting the label of the element according to the numerical value of ctheta and phi
#NOTE: BOTH will have cos(theta) = [-1,1], regardless from the sorting
cosphi_th_cos=(dfind.sort_values(by=["ctheta", "phi"]))["item"].values
cosphi_th_phi=(dfind.sort_values(by=["phi","ctheta"]))["item"].values

In [ ]:
# find the photon rotation vakues of a specific set of rotation
# where b1 is ma or min!
dfind.sort_values(by=["phi", "ctheta"])
dfind.loc[dfind["item"] == "4_7"]

In [ ]:
plt.imshow(counts_1_1.T,extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], aspect=180)
ax = plt.axes()
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
ax.set_title("MFPAD S enanant 1_1")
plt.colorbar()

In [ ]:
fig1, ax = plt.subplots(constrained_layout=True)

cs = ax.contourf(phiM,cosM,counts_1_1.T, levels=20, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], vmin=0.0001)
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
ax.set_title("MFPAD S enanant 1_1")
fig1.colorbar(cs)

In [ ]:
# convert the counts into a 2D np array and sort according to cos and phi
#NOTE: theta is along the z_MF and phi is atan(x_MF,y_MF)
counts_temp=[] #standard cos sorted S enantiomer
MFPAD_R_phi=[]
MFPAD_R_cos=[]

#NOTE the values of each MFPAD is espressend with phi[-180,180], cos(theta)[1,-1] for ELECTRONS
#counts_temp: in photon_coordiantes cos(theta)=[1,-1], phi=[-180,180] ordered by cos(theta)         
for group in frame_srt:
    #NOTE it is sorting the MFPAD values ascending phi[-180,180], cos(theta)[1,-1]
    #NOTE use phiMMS and cosMMs after this
    # elsorted=group[1].sort_values(by=["cos(theta)","phi"]) 
    # counts_temp.append(elsorted["value"].values.reshape(200,100))
    counts_temp.append(group[1]["value"].values.reshape(200,100))
    # counts_temp.append(group[0]) #equal to cosphi_th
counts_temp=np.array(counts_temp)

#MFPAD_S_phi: in photon_coordiantes cos(theta)=[-1,1], phi=[-180,180] ordered by phi
MFPAD_S_phi=fugi.sorting_array(counts_temp, theory=True, items=cosphi_th, a=2)
#MFPAD_S_cos: in photon_coordiantes cos(theta)=[-1,1], phi=[-180,180] ordered by cos
MFPAD_S_cos=fugi.sorting_array(counts_temp, theory=True, items=cosphi_th, a=1)

"""
Whith this sorting:
MFPAD_S_phi == MFPAD_S_cos gives True for the first and the last element
MFPAD_S_phi or MFPAD_S_phi == counts_temp are False, because the cos(theta) is [-1,1] compare to [1,-1]
"""
#integration of MFPAD onto cos or phi for S enantiomer IN MF
cos_projthS_phi=fugi.projection(MFPAD_S_phi,alongaxis=0)
cos_projthS_cos=fugi.projection(MFPAD_S_cos,alongaxis=0)
phi_projthS_phi=fugi.projection(MFPAD_S_phi,alongaxis=1)
phi_projthS_cos=fugi.projection(MFPAD_S_cos,alongaxis=1)

#flipping for R enantiomer and sorting
for j,el in enumerate(MFPAD_S_phi):
    tempcos_phi=np.flip(MFPAD_S_phi[j], axis=0)
    MFPAD_R_phi.append(np.flip(tempcos_phi, axis=1))

    tempcos_cos=np.flip(MFPAD_S_cos[j], axis=0)
    MFPAD_R_cos.append(np.flip(tempcos_cos, axis=1))

#integration of of MFPAD onto cos or phi for R enantiomer
cos_projthR_phi=fugi.projection(MFPAD_R_phi, alongaxis=0)
cos_projthR_cos=fugi.projection(MFPAD_R_cos, alongaxis=0)
phi_projthR_phi=fugi.projection(MFPAD_R_phi, alongaxis=1)
phi_projthR_cos=fugi.projection(MFPAD_R_cos, alongaxis=1)

#Sum in indepedent of the sorting, it is a total sum:
# MFPAD_R_phi_sum is useless!
# np.any(MFPAD_S_cos_sum == MFPAD_S_phi_sum) TRUE
MFPAD_S_cos_sum=np.sum(MFPAD_S_cos,axis=0)
MFPAD_S_phi_sum=np.sum(MFPAD_S_phi,axis=0)
MFPAD_R_cos_sum=np.sum(MFPAD_R_cos,axis=0)
# MFPAD_R_phi_sum=np.sum(MFPAD_R_phi,axis=0)

print(np.array(MFPAD_S_phi).shape)
print(np.array(MFPAD_S_cos).shape)
print(np.array(MFPAD_R_phi).shape)
print(np.array(MFPAD_R_cos).shape)
print(np.array(cos_projthS_phi).shape)
print(np.array(cos_projthR_phi).shape)

In [ ]:
# ###############norm might not be needed: already normalised by Philipp ######################
# MFPAD_S_cos_norm=fugi.normalise_matrix(MFPAD_S_cos,normtype=2)
# MFPAD_R_cos_norm=fugi.normalise_matrix(MFPAD_R_cos,normtype=2)

# MFPAD_S_cos_norm=np.array(MFPAD_S_cos_norm)
# MFPAD_R_cos_norm=np.array(MFPAD_R_cos_norm)

# MFPAD_S_cos_norm_sum=np.sum(MFPAD_S_cos_norm,axis=0)
# MFPAD_R_cos_norm_sum=np.sum(MFPAD_R_cos_norm,axis=0)
# ##############################################################################

#//////////////////////////// mPECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#up to now pretty useless
# mPECD_cos=np.divide(np.subtract(MFPAD_R_cos,MFPAD_S_cos),np.add(MFPAD_R_cos,MFPAD_S_cos))
# mPECD_phi=np.divide(np.subtract(MFPAD_R_phi,MFPAD_S_phi),np.add(MFPAD_R_phi,MFPAD_S_phi))

#//////////////////////////// cos theta PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
"""
The difference btw cos and phi lays in whitch xgo coordiantes to use when plotting the b1
The error computation is somehow unconventional, but let's normalise the error with the square root of all counts
NOTE: the order (R-S or S-R) seems to have no influence on b1 map. ?? to verify
"""
cPECD_cos =  np.divide(np.subtract(cos_projthR_cos,cos_projthS_cos),np.add(cos_projthR_cos,cos_projthS_cos))
errPECD_cos = np.sqrt(1/(cos_projthR_cos+cos_projthS_cos))

cPECD_phi =  np.divide(np.subtract(cos_projthR_phi,cos_projthS_phi),np.add(cos_projthR_phi,cos_projthS_phi))
errPECD_phi = np.sqrt(1/(cos_projthR_phi+cos_projthS_phi))

#//////////////////////////// CONTRAST MAP \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#it is a sum therefore not influenced by the sorting
mPECD_cos_sum=np.divide(np.subtract(MFPAD_R_cos_sum,MFPAD_S_cos_sum),np.add(MFPAD_R_cos_sum,MFPAD_S_cos_sum))

In [ ]:
theta_rad

In [ ]:
MFPAD_R_phi_LF=[]
MFPAD_R_cos_LF=[]

#rotation of the MFPAD according to the photon rotation and triangulating the result on a ADJUSTED meshgrid phiM, cosMM
#IS THE GRID ACCORDING TO THE COS(THETA) PHI OF THE ORIGINAL? I.E. y= 1,-1, x= -180,180
MFPAD_S_phi_LF,cos_S_phi_LF,phi_S_phi_LF=fugi.rot3d_MFPAD(MFPAD_S_phi,theta_rad,phi_rad,cosphi_adj_phi,phiMM,cosMM,method="linear")
MFPAD_S_cos_LF,cos_S_cos_LF,phi_S_cos_LF=fugi.rot3d_MFPAD(MFPAD_S_cos,theta_rad,phi_rad,cosphi_adj_cos,phiMM,cosMM,method="linear")

# MFPAD_R_phi_LF,cos_R_phi_LF,phi_R_phi_LF=fugi.rot3d_MFPAD(MFPAD_R_phi,theta_rad,phi_rad,cosphi_adj_phi,phiMM,cosMM,method="linear")
# MFPAD_R_cos_LF,cos_R_cos_LF,phi_R_cos_LF=fugi.rot3d_MFPAD(MFPAD_R_cos,theta_rad,phi_rad,cosphi_adj_cos,phiMM,cosMM,method="linear")

#flipping sorted S enantiomer for R enantiomer
for el in MFPAD_S_phi_LF:
    tempcos_phi=np.flip(el, axis=0)
    MFPAD_R_phi_LF.append(np.flip(tempcos_phi, axis=1))

for el in MFPAD_S_cos_LF:
    tempcos_cos=np.flip(el, axis=0)
    MFPAD_R_cos_LF.append(np.flip(tempcos_cos, axis=1))

#integration of MFPAD onto cos  or phi for S enantiomer
cosx_S_phi_LF=fugi.projection(MFPAD_S_phi_LF, alongaxis=0)
cosx_S_cos_LF=fugi.projection(MFPAD_S_cos_LF, alongaxis=0)
phi_projthS_phi_LF=fugi.projection(MFPAD_S_phi_LF, alongaxis=1)
phi_projthS_cos_LF=fugi.projection(MFPAD_S_cos_LF, alongaxis=1)

cosx_R_phi_LF=fugi.projection(MFPAD_R_phi_LF, alongaxis=0)
cosx_R_cos_LF=fugi.projection(MFPAD_R_cos_LF, alongaxis=0)
phi_projthR_phi_LF=fugi.projection(MFPAD_R_phi_LF, alongaxis=1)
phi_projthR_cos_LF=fugi.projection(MFPAD_R_cos_LF, alongaxis=1)

#Sum in indepedent of the sorting, it is a total sum
# np.any(MFPAD_S_cos_LF_sum == MFPAD_S_phi_LF_sum) TRUE
MFPAD_S_cos_LF_sum=np.sum(MFPAD_S_cos_LF,axis=0)
MFPAD_S_phi_LF_sum=np.sum(MFPAD_S_phi_LF,axis=0)
MFPAD_R_cos_LF_sum=np.sum(MFPAD_R_cos_LF,axis=0)


#//////////////////////////// PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#up to now pretty useless
# mPECD_LF_cos=np.divide(np.subtract(MFPAD_R_cos_LF,MFPAD_S_cos_LF),np.add(MFPAD_R_cos_LF,MFPAD_S_cos_LF))
# mPECD_LF_phi=np.divide(np.subtract(MFPAD_R_phi_LF,MFPAD_S_phi_LF),np.add(MFPAD_R_phi_LF,MFPAD_S_phi_LF))

# #//////////////////////////// cos theta PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
"""
The difference btw cos and phi lays in whitch xgo coordiantes to use when plotting the b1
The error computation is somehow unconventional, but let's normalise the error with the square root of all counts
NOTE: the order (R-S or S-R) seems to have no influence on b1 map. ?? to verify
"""
cPECD_LF_cos = np.nan_to_num(np.divide(np.subtract(cosx_R_cos_LF,cosx_S_cos_LF),np.add(cosx_R_cos_LF,cosx_S_cos_LF)))
errPECD_LF_cos = np.nan_to_num(np.sqrt(1/(cosx_R_cos_LF+cosx_S_cos_LF)))

cPECD_LF_phi = np.nan_to_num(np.divide(np.subtract(cosx_R_phi_LF,cosx_S_phi_LF),np.add(cosx_R_phi_LF,cosx_S_phi_LF)))
errPECD_LF_phi = np.nan_to_num(np.sqrt(1/(cosx_R_phi_LF+cosx_S_phi_LF)))

#//////////////////////////// CONTRAST MAP \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#it is a sum therefore not influenced by the sorting
mPECD_LF_cos_sum=np.divide(np.subtract(MFPAD_R_cos_LF_sum,MFPAD_S_cos_LF_sum),np.add(MFPAD_R_cos_LF_sum,MFPAD_S_cos_LF_sum))

#test of similarities
#the first and the last one are the same
fail1=[]
fail2=[]
for el1, el2 in zip(MFPAD_S_phi_LF,MFPAD_S_cos_LF):
    if np.any(el1!=el2): #np.any is FUNDAMENTAL
        fail1.append(el1)
        fail2.append(el2)

print(np.array(fail1).shape)
print(np.array(fail2).shape)

plt.plot(phi_full.reshape(-1),".")
plt.plot(phi_S_phi_LF[4].reshape(-1),"o")

plt.plot(ctheta_full.reshape(-1),".")
plt.plot(cos_S_phi_LF[4].reshape(-1),"o")

fig1, ax = plt.subplots(1,2, figsize=(10,5), constrained_layout=True)
ax[0].scatter(phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1),marker=".",s=1)
ax[1].scatter(phiMM,cosMM,marker=".",s=1)

#too intesive on the CPU
from scipy import interpolate

i2d = interpolate.interp2d(phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1),MFPAD_S_phi_LF[4].reshape(-1))
znew = i2d(phiM.reshape(-1), cosM.reshape(-1))

import matplotlib.tri as tri 
triang = tri.Triangulation(phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1))
interpolator = tri.CubicTriInterpolator(triang, MFPAD_S_phi_LF[4].reshape(-1), kind='geom')
phiMM_test, cosMM_test = np.meshgrid(phiM, cosM)
zinterp = interpolator(phiMM_test,cosMM_test)
#zinterp = tri.interp_cubic_geom(phiM,cosM)

#even with 4 subdivided triangles
#refiner = tri.UniformTriRefiner(triang)
#tri_refi, z_test_refi = refiner.refine_field(MFPAD_S_phi_LF[4].reshape(-1), subdiv=1)

#ax[0,1].contourf(phiMM,cosMM,zinterp)
#ax[0,1].set_xlabel('phi [DEG]')
#ax[0,1].set_ylabel('cos(theta) [adm]')
#ax[0,1].set_title("interp2d rot")


#equivalente to interpolator methoh
#to be used after the 3D rotation of the MFPAD passing through cartesian coordiantes
from scipy.interpolate import griddata
zi = griddata((phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1)), MFPAD_S_phi_LF[4].reshape(-1), (phiMM, cosMM), method='linear')
zi_zero = np.nan_to_num(zi,np.nan)


In [ ]:
#TEST: rotation in 2D!
fugi = reload(fugi)
tMFPAD_R_cos_LF=[]

tMFPAD_S_cos_LF,tcos_S_cos_LF,tphi_S_cos_LF=fugi.rot2d_MFPAD(MFPAD_S_cos,ctheta_full,phi_full,cosphi_adj_cos,phiMM,cosMM,method="linear")
tMFPAD_S_cos_LF=(np.nan_to_num(tMFPAD_S_cos_LF,np.nan)).reshape(72,200,100)

for el in tMFPAD_S_cos_LF:
    tempcos_cos=np.flip(el, axis=0)
    tMFPAD_R_cos_LF.append(np.flip(tempcos_cos, axis=1))

tcosx_S_cos_LF=fugi.projection(tMFPAD_S_cos_LF, alongaxis=0)
tcosx_R_cos_LF=fugi.projection(tMFPAD_R_cos_LF, alongaxis=0)

tcPECD_LF_cos = np.nan_to_num(np.divide(np.subtract(tcosx_R_cos_LF,tcosx_S_cos_LF),np.add(tcosx_R_cos_LF,tcosx_S_cos_LF)))
terrPECD_LF_cos = np.nan_to_num(np.sqrt(1/(tcosx_R_cos_LF+tcosx_S_cos_LF)))

tMFPAD_S_cos_LF_sum=np.sum(tMFPAD_S_cos_LF,axis=0)
tMFPAD_R_cos_LF_sum=np.sum(tMFPAD_R_cos_LF,axis=0)

tmPECD_LF_cos=np.divide(np.subtract(tMFPAD_R_cos_LF,tMFPAD_S_cos_LF),np.add(tMFPAD_R_cos_LF,tMFPAD_S_cos_LF))
tmPECD_LF_cos_sum=np.divide(np.subtract(tMFPAD_R_cos_LF_sum,tMFPAD_S_cos_LF_sum),np.add(tMFPAD_R_cos_LF_sum,tMFPAD_S_cos_LF_sum))


In [ ]:
#NOTE: after the triangluation one has to use phiMM and cosMM
fig1, ax = plt.subplots(1,3, figsize=(14,5), constrained_layout=True)
# fig1, ax = plt.subplots(constrained_layout=True)

# ax[0].contourf(triang,zinterp.reshape(-1))
# ax[0].contourf(phiMM,cosMM,MFPAD_S_cos_LF[4].T)
ax[0].contourf(phiMM,cosMM,MFPAD_S_cos[0].T)
# ax[0].contourf(phiMM,cosMM,MFPAD_S_cos_LF_norm_sum.T)
# ax[0].contourf(phiMM, cosMM, MFPAD_temp)
# ax[0].contourf(znew.T)
ax[0].set_xlabel('phi [DEG]')
ax[0].set_ylabel('cos(theta) [adm]')
ax[0].set_title("original")

ax[1].contourf(phiMM, cosMM, tMFPAD_S_cos_LF[0].T)
# ax[1].tricontourf(phi_full.reshape(-1), ctheta_full.reshape(-1), tMFPAD_S_cos_LF[4].reshape(-1))
# ax[1].tricontourf(tphi_S_cos_LF[4].reshape(-1), tcos_S_cos_LF[4].reshape(-1), tMFPAD_S_cos_LF[4].reshape(-1))
# ax[1].contourf(phiMM, cosMM, test[4].T)
# ax[1].tricontourf(tphi_S_cos_LF[4].reshape(-1), tcos_S_cos_LF[4].reshape(-1), tMFPAD_S_cos_LF[4].reshape(-1))
# ax[1].contourf(tphi_S_cos_LF[4], tcos_S_cos_LF[4], tMFPAD_S_cos_LF[4])
# ax[1].contourf(phi_S_cos_LF[4], cos_S_cos_LF[4], MFPAD_S_cos_LF[4])
# ax[1].contourf(phiM, cosM, tMFPAD_S_cos_LF[4].reshape(200,100).T)
ax[1].set_xlabel('phi [DEG]')
ax[1].set_ylabel('cos(theta) [adm]')
ax[1].set_title("rot2D in sperical")

ax[2].contourf(phiMM,cosMM,MFPAD_S_cos_LF[0].T)
ax[2].set_xlabel('phi [DEG]')
ax[2].set_ylabel('cos(theta) [adm]')
ax[2].set_title("rot3D with cartesian")

fig1.savefig('../PYTHON_graphs/OUTPUTS/Till_rotaton.png', figsize=(14,5), dpi=144, constrained_layout=True, transparent=False)

from scipy.interpolate import griddata

ctheta_rot=[];phi_rot=[];
for cth,ph in zip(ctheta_full.reshape(-1),phi_full.reshape(-1)):
    tempct=cth+cosphi_adj[0][0]
    if tempct<-1.:
        ctheta_rot.append(tempct+2.)
    elif tempct>=1.:
        ctheta_rot.append(tempct-2.)
    else:
        ctheta_rot.append(tempct)

    tempp=ph+cosphi_adj[0][1]
    if tempp<-180.:
        phi_rot.append(tempp+360.)
    elif tempp>=180.:
        phi_rot.append(tempp-360.)
    else:
        phi_rot.append(tempp)


MFPAD_temp=griddata(list(zip(phi_rot,ctheta_rot)), MFPAD_S_cos[0].reshape(-1), (phiMM.T, cosMM.T), method="linear")

MFPAD_temp_new=griddata((phi_full.reshape(-1), ctheta_full.reshape(-1)), MFPAD_S_cos[0].reshape(-1), (np.array(phi_rot).reshape(200,100),np.array(ctheta_rot).reshape(200,100)), method="linear")

MFPAD_temp_new1=griddata((cosMM.reshape(-1),phiMM.reshape(-1)), MFPAD_S_cos[0].reshape(-1), (np.array(ctheta_rot).reshape(200,100),np.array(phi_rot).reshape(200,100)), method="linear")

MFPAD_temp_new2=griddata((phiMM.reshape(-1), cosMM.reshape(-1)), MFPAD_S_cos[0].reshape(-1), (np.array(phi_rot).reshape(100,200),np.array(ctheta_rot).reshape(100,200)), method="linear")

plt.contourf(np.array(phi_rot).reshape(200,100), np.array(ctheta_rot).reshape(200,100), MFPAD_temp_new)

In [ ]:
# all the 72 plots S ENANTIOMER MOLECULAR FRAME
# fugi = reload(fugi)
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),tMFPAD_S_phi_LF[i].reshape(-1))
    # ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_phi[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.contourf(phiMM,cosMM,MFPAD_S_phi[i].T)
    ax.tick_params(labelsize=12)
    ax.text(0, 0, "S_en "+str(cosphi_th_phi[i]), ha="center", va="center", color="red",fontsize=18)
    # ax.text(0, 0, str(cosphi_adj_cos[i]), ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig2a)
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_S_phi.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
# all the 72 plots S ENANTIOMER MOLECULAR FRAME
# fugi = reload(fugi)
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),tMFPAD_S_phi_LF[i].reshape(-1))
    # ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_S_phi[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.contourf(phiMM,cosMM,MFPAD_S_cos[i].T)
    ax.tick_params(labelsize=12)
    ax.text(0, 0, "S_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)
    # ax.text(0, 0, str(cosphi_adj_cos[i]), ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig2a)
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_S_cos.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
# all the 72 plots S ENANTIOMER LABORATORY FRAME 2D ROTATED
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
#6 levels of cos(theta) (y), 12 levels of phi (x)

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),tMFPAD_S_phi_LF[i].reshape(-1))
    # ax.contourf(phiMM,cosMM,MFPAD_S_cos_LF[i].T)
    ax.imshow(MFPAD_S_cos_LF[i].T, extent=(-180,180,-1,1), aspect = 180, origin='lower')
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, "S_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)
    ax.text(0, 0, str(cosphi_adj_cos[i]), ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig2a)
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_S_cos_LF_with axes.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()    

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
# all the 72 plots FOR R ENANTIOMER NOT SORTED: to be compare directly with experimental data
#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.flatten()) :
    # ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),MFPAD_R_cos[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.contourf(phiMM,cosMM,MFPAD_R_cos[i].T)
    ax.tick_params(labelsize=12)    
    ax.text(0, 0, "R_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)
    # ax.text(0, 0, str(cosphi_adj_cos[i]), ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig2a)
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_R_cos.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
# all the 72 plots S ENANTIOMER LABORATORY FRAME 2D ROTATED
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
#6 levels of cos(theta) (y), 12 levels of phi (x)

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),tMFPAD_S_phi_LF[i].reshape(-1))
    ax.contourf(phiMM,cosMM,MFPAD_R_cos_LF[i].T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, "R_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)
    ax.text(0, 0, str(cosphi_adj_cos[i]), ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig2a)
fig2a.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_R_cos_LF_with axes.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()    

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
# all the 72 plots S ENANTIOMER LABORATORY FRAME 2D ROTATED
#the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
#6 levels of cos(theta) (y), 12 levels of phi (x)

#fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.flatten()) :
    # ax.tricontourf(phi_S_phi_LF[i].reshape(-1),cos_S_phi_LF[i].reshape(-1),tMFPAD_S_phi_LF[i].reshape(-1))
    ax.contourf(phiMM,cosMM,tMFPAD_S_cos_LF[i].T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, "S_en "+str(cosphi_th_cos[i]), ha="center", va="center", color="red",fontsize=18)
    ax.text(0, 0, str(cosphi_adj_cos[i]), ha="center", va="center", color="red",fontsize=12)

fugi.overlaygraph(fig2a)
fig2a.savefig('../PYTHON_graphs/OUTPUTS/tMFPAD_S_cos_LF_with axes.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()    

#WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
#for i,ax in zip(range(72),axes.T.flatten()) :

In [ ]:
#PECD for non rotated
plt.figure(figsize=(3, 3))
plt.plot(errPECD_cos[60],".")
plt.plot(cPECD_cos[60])
plt.plot(cos_projthR_cos[60])
plt.plot(cos_projthS_cos[60])
print(np.sum(cos_projthR_cos[60]))

In [ ]:
#PECD for non rotated
plt.figure(figsize=(3, 3))
plt.plot(errPECD_LF_cos[60],".")
plt.plot(cPECD_LF_cos[60])
plt.plot(cosx_R_cos_LF[60])
plt.plot(cosx_S_cos_LF[60])
print(np.sum(cosx_R_cos_LF[60]))

In [ ]:
#comparison of PECD with different cos(theta)
fig1, ax = plt.subplots(1,3, figsize=(8, 2), dpi=40, constrained_layout=True)
#along z axis
ax[0].plot(cos_projthR_cos[0])
ax[0].plot(cos_projthS_cos[0])

#along x axis
ax[1].plot(cosx_R_cos_LF[0])
ax[1].plot(cosx_S_cos_LF[0])

#along y axis
ax[2].plot(phi_projthR_cos_LF[0])
ax[2].plot(phi_projthS_cos_LF[0])

In [ ]:
import functions_minuit as fumi
#PHI ordered modified with error

# param_matrix=np.zeros((6,72)) # alternative nparray initialization
param_matrix_phi = [[0 for i in range(6)] for j in range(72)] #i col, j row

# fig, axes = plt.subplots(6,12, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(6,12, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.6,0.6) #custom_ylim = (-1,1)

limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
# limits=[(-1, 1), (-1, 1), (None, None), (None, None), (None, None), (None, None)]

for i,ax in zip(range(72),axes.flatten()): #fixe because of the photon directions, equal to range(len(cosphi))
    #NOTE: x_data has to be monotonicalli INCREASING!
    x_data = np.flip(cosM);y_data = cPECD_LF_phi[i];
    #original in MF
    # x_data = np.flip(cosM);y_data = cPECD_phi[i];y_err = errPECD_phi[i]

    m=fumi.iminuit_noerr(x_data,y_data,limits)
    fumi.plotminuit_noerr(x_data,y_data,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_phi[i][counter]=((m.values[el], m.errors[el]))

param_matrix_phi=np.array(param_matrix_phi) #numpy tranformation!!!

fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
fig.savefig('../PYTHON_graphs/OUTPUTS/b1matrix_R-S_F_cos_theory.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show() 

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(25, 10))
cs1=ax1.contourf(param_matrix_phi[:,0,0].reshape(6,12), cmap=cmap_temp, levels=15)
ax1.set_title("MFPAD SCL")
fig.colorbar(cs1,extend='both', shrink=0.9, ax=ax1)

cs2=ax2.tricontourf(np.array(xgo_phi).reshape(-1),np.array(ygo_phi).reshape(-1),param_matrix_phi[:,0,0].reshape(-1),cmap=cmap_temp,levels=15)
ax2.set_title("MFPAD SCL")
fig.colorbar(cs2,extend='both', shrink=0.9, ax=ax2)
plt.show()

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('b1 map Without Smoothing', 'With Smoothing'))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_phi, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_phi[:,0,0], x=xgo_phi, y=ygo_phi, line_smoothing=0, colorscale=cmap_temp_go), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_phi[:,0,0], x=xgo_phi, y=ygo_phi, line_smoothing=0.75, colorscale=cmap_temp_go), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
#to be compared with the b1 maps of the experimental data AS THEY ARE (not sorted)
fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(6):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=param_matrix_phi[:,i,0], x=xgo_phi, y=ygo_phi, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_phi[:,i,0], x=xgo_phi, y=ygo_phi, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=param_matrix_phi[:,i,0], x=xgo_phi, y=ygo_phi, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_phi[:,i,0], x=xgo_phi, y=ygo_phi, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
#cos ordered modified with error
fumi = reload(fumi)

# param_matrix=np.zeros((6,72)) # alternative nparray initialization
param_matrix_cos = [[0 for i in range(6)] for j in range(72)] #i col, j row

# fig, axes = plt.subplots(6,12, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(6,12, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.6,0.6) #custom_ylim = (-1,1)

limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
# limits=[(-1, 1), (-1, 1), (None, None), (None, None), (None, None), (None, None)]

for i,ax in zip(range(72),axes.flatten()): #fixe because of the photon directions, equal to range(len(cosphi))
    #NOTE: x_data has to be monotonicalli INCREASING!
    x_data = np.flip(cosM);y_data = cPECD_LF_cos[i];
    # x_data = np.flip(cosM);y_data = cPECD_LF_cos[i];y_err = errPECD_LF_cos[i]
    #original in MF
    # x_data = np.flip(cosM);y_data = cPECD_cos[i];y_err = errPECD_cos[i]

    m=fumi.iminuit_noerr(x_data,y_data,limits)
    fumi.plotminuit_noerr(x_data,y_data,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_cos[i][counter]=((m.values[el], m.errors[el]))

param_matrix_cos=np.array(param_matrix_cos) #numpy tranformation!!!

fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
fig.savefig('../PYTHON_graphs/OUTPUTS/b1matrix_R-S_LF_cos_theory.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show() 

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('b1 map Without Smoothing', 'With Smoothing'))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_cos, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[:,0,0], x=xgo_cos, y=ygo_cos, line_smoothing=0, colorscale=cmap_temp_go), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[:,0,0], x=xgo_cos, y=ygo_cos, line_smoothing=0.75, colorscale=cmap_temp_go), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
#to be compared with the b1 maps of the experimental data AS THEY ARE (not sorted)
fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(6):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=param_matrix_cos[:,i,0], x=xgo_cos, y=ygo_cos, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_cos[:,i,0], x=xgo_cos, y=ygo_cos, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=param_matrix_cos[:,i,0], x=xgo_cos, y=ygo_cos, line_smoothing=0, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_cos[:,i,0], x=xgo_cos, y=ygo_cos, line_smoothing=0.85, colorscale=cmap_temp_go, legendgroup="yes", contours=dict(start=-0.5, end=0.5, size=0.1)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
#before rotation, in the MF
#cos ordered modified with error

# param_matrix=np.zeros((6,72)) # alternative nparray initialization
param_matrix_cos = [[0 for i in range(6)] for j in range(72)] #i col, j row

# fig, axes = plt.subplots(6,12, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(6,12, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.6,0.6) #custom_ylim = (-1,1)

limits=[(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
# limits=[(-1, 1), (-1, 1), (None, None), (None, None), (None, None), (None, None)]

for i,ax in zip(range(72),axes.flatten()): #fixe because of the photon directions, equal to range(len(cosphi))
    #NOTE: x_data has to be monotonicalli INCREASING!
    # x_data = np.flip(cosM);y_data = cPECD_LF_cos[i];y_err = errPECD_LF_cos[i]
    #original in MF
    x_data = np.flip(cosM);y_data = cPECD_cos[i];

    m=fumi.iminuit_noerr(x_data,y_data,limits)
    fumi.plotminuit_noerr(x_data,y_data,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): # len = 6
        param_matrix_cos[i][counter]=((m.values[el], m.errors[el]))

param_matrix_cos=np.array(param_matrix_cos) #numpy tranformation!!!

fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)
fig.savefig('../PYTHON_graphs/OUTPUTS/b1matrix_R-S_MF_cos_theory.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show() 

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    x_title='phi_photon [DEG]',
                    y_title='cos(theta) [adm]',
                    subplot_titles=('b1 map Without Smoothing', 'With Smoothing'))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_cos, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[:,0,0], x=xgo_cos, y=ygo_cos, line_smoothing=0, colorscale=cmap_temp_go), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[:,0,0], x=xgo_cos, y=ygo_cos, line_smoothing=0.75, colorscale=cmap_temp_go), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
#INTEGRATED S AND R MFPADs in MF

fig, ax = plt.subplots(3,2,sharex=True,sharey=True,figsize=(12, 18))

ax[0,0].contourf(phiM,cosM,MFPAD_R_cos_sum.reshape(200,100).T,cmap=cmap_temp)
ax[0,0].set_title("MFPAD R MF")
ax[0,0].set_ylabel('cos(theta) [adm]')

ax[0,1].contourf(phiM,cosM,MFPAD_S_cos_sum.reshape(200,100).T,cmap=cmap_temp)
ax[0,1].set_title("MFPAD S MF")

ax[1,0].contourf(phiMM,cosMM,MFPAD_R_cos_LF_sum.T,cmap=cmap_temp, levels=20)
ax[1,0].set_title("MFPAD R LF")
ax[1,0].set_ylabel('cos(theta) [adm]')

ax[1,1].contourf(phiMM,cosMM,MFPAD_S_cos_LF_sum.T,cmap=cmap_temp,vmin=0.08,vmax=0.175, levels=20)
ax[1,1].set_title("MFPAD S LF")

ax[2,0].contourf(phiMM,cosMM,MFPAD_R_cos_LF_sum.T, cmap=cmap_temp, levels=20)
ax[2,0].set_title("MFPAD S LF norm")
ax[2,0].set_ylabel('cos(theta) [adm]')
ax[2,0].set_xlabel('phi [DEG]')

ax[2,1].contourf(phiMM,cosMM,MFPAD_S_cos_LF_sum.T, cmap=cmap_temp,vmin=4.7, vmax=8.3, levels=20)
ax[2,1].set_title("MFPAD S LF norm")
ax[2,1].set_xlabel('phi [DEG]')
plt.show()

In [ ]:
#to compare with contrast map!
#is the sum the same?

# interpolation: [ 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'.]

fig0=plt.figure()
plt.contourf(phiMM,cosMM,mPECD_cos_sum.T, vmin=-0.3, vmax=0.3, cmap=cmap_temp)
# plt.imshow(mPECD_cos_sum, vmin=-0.3, vmax=0.3, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='none', cmap=cmap_temp, aspect="auto")
plt.title("Contrast map theory")
plt.xlabel('phi [DEG]')
plt.ylabel('cos(theta) [adm]')
plt.colorbar()

# fig1=plt.figure()
# plt.imshow(mPECD_LF_cos_sum, vmin=-0.22, vmax=0.22, origin="lower", extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap=cmap_temp, aspect="auto")
# plt.title("imshow rotated quadratic")
# plt.xlabel('phi [DEG]')
# plt.ylabel('cos(theta) [adm]')
# plt.colorbar()

fig2=plt.figure()
plt.contourf(phiMM,cosMM, mPECD_LF_cos_sum.T, vmin=-0.2, vmax=0.2, levels= 40, cmap=cmap_temp)
plt.title("contourf 3D rotated norm")
plt.xlabel('phi [DEG]')
plt.ylabel('cos(theta) [adm]')
plt.colorbar()

# fig3=plt.figure()
# # plt.imshow(tmPECD_cos_sum, vmin=-0.2, vmax=0.2, origin="lower", extent=[phiM[0],phiM[-1],cosM[-1],cosM[0]], interpolation='quadric', cmap=cmap_temp, aspect="auto")
# plt.contourf(phiM,cosM, tmPECD_LF_cos_sum.T, vmin=-0.2, vmax=0.2, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], levels= 40, cmap=cmap_temp)
# plt.title("contourf 2D rotated")
# plt.xlabel('phi [DEG]')
# plt.ylabel('cos(theta) [adm]')
# plt.colorbar()